In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import json
import os

class ToDoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("To-Do List Application")
        self.root.geometry("800x600")
        self.tasks = []
        self.current_filter = "All"
        
        # Create data file if it doesn't exist
        self.data_file = "todo_data.json"
        if not os.path.exists(self.data_file):
            with open(self.data_file, 'w') as f:
                json.dump([], f)
        
        self.load_tasks()
        self.setup_ui()
    
    def setup_ui(self):
        # Main container
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Input section
        input_frame = ttk.LabelFrame(main_frame, text="Add New Task", padding="10")
        input_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Task description
        ttk.Label(input_frame, text="Task:").grid(row=0, column=0, sticky=tk.W)
        self.task_entry = ttk.Entry(input_frame, width=50)
        self.task_entry.grid(row=0, column=1, padx=5, pady=5, sticky=tk.EW)
        
        # Category
        ttk.Label(input_frame, text="Category:").grid(row=1, column=0, sticky=tk.W)
        self.category_var = tk.StringVar()
        self.category_combobox = ttk.Combobox(
            input_frame, 
            textvariable=self.category_var,
            values=["Personal", "Work", "Shopping", "Other"]
        )
        self.category_combobox.grid(row=1, column=1, padx=5, pady=5, sticky=tk.EW)
        self.category_combobox.set("Personal")
        
        # Priority
        ttk.Label(input_frame, text="Priority:").grid(row=2, column=0, sticky=tk.W)
        self.priority_var = tk.StringVar()
        priority_frame = ttk.Frame(input_frame)
        priority_frame.grid(row=2, column=1, padx=5, pady=5, sticky=tk.W)
        ttk.Radiobutton(priority_frame, text="High", variable=self.priority_var, value="High").pack(side=tk.LEFT)
        ttk.Radiobutton(priority_frame, text="Medium", variable=self.priority_var, value="Medium").pack(side=tk.LEFT, padx=5)
        ttk.Radiobutton(priority_frame, text="Low", variable=self.priority_var, value="Low").pack(side=tk.LEFT)
        self.priority_var.set("Medium")
        
        # Due date
        ttk.Label(input_frame, text="Due Date:").grid(row=3, column=0, sticky=tk.W)
        self.due_date_entry = ttk.Entry(input_frame)
        self.due_date_entry.grid(row=3, column=1, padx=5, pady=5, sticky=tk.W)
        self.due_date_entry.insert(0, datetime.now().strftime("%Y-%m-%d"))
        
        # Add task button
        add_button = ttk.Button(input_frame, text="Add Task", command=self.add_task)
        add_button.grid(row=4, column=1, pady=5, sticky=tk.E)
        
        # Task list section
        list_frame = ttk.LabelFrame(main_frame, text="Tasks", padding="10")
        list_frame.pack(fill=tk.BOTH, expand=True)
        
        # Filter buttons
        filter_frame = ttk.Frame(list_frame)
        filter_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(filter_frame, text="All", command=lambda: self.filter_tasks("All")).pack(side=tk.LEFT)
        ttk.Button(filter_frame, text="Pending", command=lambda: self.filter_tasks("Pending")).pack(side=tk.LEFT, padx=5)
        ttk.Button(filter_frame, text="Completed", command=lambda: self.filter_tasks("Completed")).pack(side=tk.LEFT)
        ttk.Button(filter_frame, text="High Priority", command=lambda: self.filter_tasks("High")).pack(side=tk.LEFT, padx=5)
        
        # Treeview for tasks
        self.tree = ttk.Treeview(
            list_frame,
            columns=("ID", "Task", "Category", "Priority", "Due Date", "Status"),
            show="headings",
            selectmode="browse"
        )
        
        # Configure columns
        self.tree.heading("ID", text="ID", anchor=tk.W)
        self.tree.heading("Task", text="Task", anchor=tk.W)
        self.tree.heading("Category", text="Category", anchor=tk.W)
        self.tree.heading("Priority", text="Priority", anchor=tk.W)
        self.tree.heading("Due Date", text="Due Date", anchor=tk.W)
        self.tree.heading("Status", text="Status", anchor=tk.W)
        
        self.tree.column("ID", width=50, stretch=tk.NO)
        self.tree.column("Task", width=200)
        self.tree.column("Category", width=100)
        self.tree.column("Priority", width=100)
        self.tree.column("Due Date", width=100)
        self.tree.column("Status", width=100)
        
        self.tree.pack(fill=tk.BOTH, expand=True)
        
        # Scrollbar
        scrollbar = ttk.Scrollbar(self.tree, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Action buttons
        button_frame = ttk.Frame(list_frame)
        button_frame.pack(fill=tk.X, pady=(10, 0))
        
        ttk.Button(button_frame, text="Mark Complete", command=self.mark_complete).pack(side=tk.LEFT)
        ttk.Button(button_frame, text="Edit Task", command=self.edit_task).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Delete Task", command=self.delete_task).pack(side=tk.LEFT)
        ttk.Button(button_frame, text="Refresh", command=self.refresh_tasks).pack(side=tk.RIGHT)
        
        # Populate the treeview
        self.refresh_tasks()
    
    def load_tasks(self):
        try:
            with open(self.data_file, 'r') as f:
                self.tasks = json.load(f)
        except (json.JSONDecodeError, FileNotFoundError):
            self.tasks = []
    
    def save_tasks(self):
        with open(self.data_file, 'w') as f:
            json.dump(self.tasks, f)
    
    def add_task(self):
        task_text = self.task_entry.get().strip()
        category = self.category_var.get()
        priority = self.priority_var.get()
        due_date = self.due_date_entry.get().strip()
        
        if not task_text:
            messagebox.showwarning("Warning", "Task description cannot be empty!")
            return
        
        try:
            datetime.strptime(due_date, "%Y-%m-%d")
        except ValueError:
            messagebox.showwarning("Warning", "Invalid date format! Use YYYY-MM-DD")
            return
        
        new_task = {
            "id": len(self.tasks) + 1,
            "task": task_text,
            "category": category,
            "priority": priority,
            "due_date": due_date,
            "status": "Pending",
            "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        self.tasks.append(new_task)
        self.save_tasks()
        self.refresh_tasks()
        
        # Clear input fields
        self.task_entry.delete(0, tk.END)
        self.category_combobox.set("Personal")
        self.priority_var.set("Medium")
        self.due_date_entry.delete(0, tk.END)
        self.due_date_entry.insert(0, datetime.now().strftime("%Y-%m-%d"))
    
    def refresh_tasks(self):
        # Clear the treeview
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Add tasks to the treeview
        for task in self.tasks:
            if (self.current_filter == "All" or
                (self.current_filter == "Pending" and task["status"] == "Pending") or
                (self.current_filter == "Completed" and task["status"] == "Completed") or
                (self.current_filter == "High" and task["priority"] == "High")):
                
                self.tree.insert(
                    "", tk.END,
                    values=(
                        task["id"],
                        task["task"],
                        task["category"],
                        task["priority"],
                        task["due_date"],
                        task["status"]
                    )
                )
    
    def filter_tasks(self, filter_type):
        self.current_filter = filter_type
        self.refresh_tasks()
    
    def get_selected_task(self):
        selected_item = self.tree.selection()
        if not selected_item:
            messagebox.showwarning("Warning", "Please select a task first!")
            return None
        
        item = self.tree.item(selected_item[0])
        task_id = item["values"][0]
        
        for task in self.tasks:
            if task["id"] == task_id:
                return task
        
        return None
    
    def mark_complete(self):
        task = self.get_selected_task()
        if task:
            task["status"] = "Completed"
            self.save_tasks()
            self.refresh_tasks()
    
    def delete_task(self):
        task = self.get_selected_task()
        if task:
            confirm = messagebox.askyesno("Confirm", "Are you sure you want to delete this task?")
            if confirm:
                self.tasks = [t for t in self.tasks if t["id"] != task["id"]]
                self.save_tasks()
                self.refresh_tasks()
    
    def edit_task(self):
        task = self.get_selected_task()
        if not task:
            return
        
        # Create edit window
        edit_window = tk.Toplevel(self.root)
        edit_window.title("Edit Task")
        edit_window.geometry("400x300")
        
        # Task description
        ttk.Label(edit_window, text="Task:").pack(pady=(10, 0))
        task_entry = ttk.Entry(edit_window, width=40)
        task_entry.pack()
        task_entry.insert(0, task["task"])
        
        # Category
        ttk.Label(edit_window, text="Category:").pack(pady=(10, 0))
        category_var = tk.StringVar(value=task["category"])
        category_combobox = ttk.Combobox(
            edit_window, 
            textvariable=category_var,
            values=["Personal", "Work", "Shopping", "Other"]
        )
        category_combobox.pack()
        
        # Priority
        ttk.Label(edit_window, text="Priority:").pack(pady=(10, 0))
        priority_var = tk.StringVar(value=task["priority"])
        priority_frame = ttk.Frame(edit_window)
        priority_frame.pack()
        ttk.Radiobutton(priority_frame, text="High", variable=priority_var, value="High").pack(side=tk.LEFT)
        ttk.Radiobutton(priority_frame, text="Medium", variable=priority_var, value="Medium").pack(side=tk.LEFT, padx=5)
        ttk.Radiobutton(priority_frame, text="Low", variable=priority_var, value="Low").pack(side=tk.LEFT)
        
        # Due date
        ttk.Label(edit_window, text="Due Date:").pack(pady=(10, 0))
        due_date_entry = ttk.Entry(edit_window)
        due_date_entry.pack()
        due_date_entry.insert(0, task["due_date"])
        
        # Status
        ttk.Label(edit_window, text="Status:").pack(pady=(10, 0))
        status_var = tk.StringVar(value=task["status"])
        status_combobox = ttk.Combobox(
            edit_window, 
            textvariable=status_var,
            values=["Pending", "Completed"],
            state="readonly"
        )
        status_combobox.pack()
        
        # Save button
        def save_changes():
            task["task"] = task_entry.get().strip()
            task["category"] = category_var.get()
            task["priority"] = priority_var.get()
            task["due_date"] = due_date_entry.get().strip()
            task["status"] = status_var.get()
            
            try:
                datetime.strptime(task["due_date"], "%Y-%m-%d")
            except ValueError:
                messagebox.showwarning("Warning", "Invalid date format! Use YYYY-MM-DD")
                return
            
            self.save_tasks()
            self.refresh_tasks()
            edit_window.destroy()
        
        ttk.Button(edit_window, text="Save Changes", command=save_changes).pack(pady=20)

if __name__ == "__main__":
    root = tk.Tk()
    app = ToDoApp(root)
    root.mainloop()